### From articles "Options for smart investor"

https://dzen.ru/id/5f6b2336d03ed90da6bf9d00


In [32]:
# Prepare data and option_lib to use in Google Collab
!bash -c '\
OPT_LIB_PATH="/content/option_lib"; \
if [[ ! (-z "${COLAB_JUPYTER_IP}" || -d "${OPT_LIB_PATH}" ) ]]; then \
  git clone https://github.com/akumidv/option_lib.git "${OPT_LIB_PATH}"; \
  DATA_PATH="/content/data"; \
  mkdir -p "${DATA_PATH}/DERIBIT/BTC-USD/EOD/futures"; \
  mkdir -p "${DATA_PATH}/DERIBIT/BTC-USD/EOD/options"; \
  gdown --fuzzy "https://drive.google.com/file/d/1n2T4jBHeanGLtBLlLWgYIHzK9hYsGT97/view?usp=sharing"  -O "/content/data/DERIBIT/BTC-USD/EOD/futures/2025.parquet" ;\
  gdown --fuzzy "https://drive.google.com/file/d/10Eo_4oNSlx0rbe2efn5OXToONAi5nPhV/view?usp=sharing"  -O "/content/data/DERIBIT/BTC-USD/EOD/options/2025.parquet"  ;\
  export VENV_PATH="${OPT_LIB_PATH}/.venv";\
  pip install poetry ;\
  cd ${OPT_LIB_PATH} ;\
  POETRY_VIRTUALENVS_IN_PROJECT=true poetry install --no-interaction;\
fi'
import sys, os

if os.path.isdir('/content/option_lib') and '/content/option_lib/src' not in sys.path:
    sys.path.extend(['/content/option_lib/src', '/content/option_lib/.venv/'])  # Not work yet - poetry install somewhere else
    os.environ['DATA_PATH'] = '/content/data'


In [33]:
import datetime
import pandas as pd
from chart_studio import plotly

from plotly import graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#import plotly.express as px

from option_lib.provider import PandasLocalFileProvider, RequestParameters
from option_lib.entities import OptionType, OptionPriceStatus
from option_lib import Option
from option_lib.entities import LegType, OptionLeg

init_notebook_mode(connected=True)

pd.set_option("display.max_rows", 20, "display.max_columns", 30)

In [41]:
deribit_provider = PandasLocalFileProvider('DERIBIT', os.environ.get('DATA_PATH', '../../data'))
cur_dt = datetime.date.today()
provider_params = RequestParameters(period_to=cur_dt.year)
btc = Option(deribit_provider, 'BTC', provider_params,
             option_columns=['timestamp', 'expiration_date', 'strike', 'option_type', 'price', 'underlying_expiration_date', 'exchange_price', 'exchange_iv', 'underlying_price']) #, 'iv', 'delta', 'gamma', 'vega', 'theta', 'quick_delta', 'contract_size'])


In [42]:
btc.df_hist

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price
0,2025-02-11 00:00:00+00:00,2025-02-11 00:00:00+00:00,100000.0,c,NaN,NaT,0.000000,55.06,98378.9700
1,2025-02-12 00:00:00+00:00,2025-02-11 00:00:00+00:00,100000.0,c,NaN,NaT,0.000000,55.06,98378.9700
2,2025-02-11 00:00:00+00:00,2025-02-11 00:00:00+00:00,100000.0,p,NaN,NaT,NaN,55.06,98378.9700
3,2025-02-12 00:00:00+00:00,2025-02-11 00:00:00+00:00,100000.0,p,NaN,NaT,NaN,55.06,98378.9700
4,2025-02-11 00:00:00+00:00,2025-02-11 00:00:00+00:00,101000.0,c,NaN,NaT,0.000000,56.71,98378.9700
...,...,...,...,...,...,...,...,...,...
8147,2025-02-16 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4952.94143,2025-03-07 00:00:00+00:00,5402.500208,46.74,96553.1400
8148,2025-02-17 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,5459.36082,2025-03-07 00:00:00+00:00,5608.945391,46.71,96022.4200
8149,2025-02-18 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,5449.31514,2025-03-07 00:00:00+00:00,5614.190384,46.80,95843.3500
8150,2025-02-19 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,5508.17448,2025-03-07 00:00:00+00:00,4575.306185,42.69,96872.1700


In [8]:
df_fut = btc.df_fut
df_fut.head(2)

,timestamp,expiration_date,price
0,2025-02-11 00:00:00+00:00,2025-02-14 00:00:00+00:00,95800.0
1,2025-02-12 00:00:00+00:00,2025-02-14 00:00:00+00:00,97942.5


In [9]:
btc.enrichment.add_future().add_intrinsic_and_time_value()
btc.df_hist.head(2)

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value
96,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.575306,2025-02-12 00:00:00+00:00,5.254928,60.36,95757.81054,0.0,9.575306
97,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.604906,2025-02-12 00:00:00+00:00,0.000000,62.78,96049.03000,0.0,9.604906


In [10]:
btc.enrichment.add_atm_itm_otm()
btc.df_hist[btc.df_hist['price_status'] == OptionPriceStatus.ATM.code].head(2)

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
208,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,96000.0,c,526.641830,2025-02-12 00:00:00+00:00,338.273538,38.81,95757.81054,0.00,526.641830,atm
209,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,96000.0,c,57.629436,2025-02-12 00:00:00+00:00,50.398863,27.13,96049.03000,49.03,8.599436,atm


In [11]:
df_opt = btc.df_hist
df_opt

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
96,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.575306,2025-02-12 00:00:00+00:00,5.254928,60.36,95757.81054,0.00000,9.575306,otm
97,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.604906,2025-02-12 00:00:00+00:00,0.000000,62.78,96049.03000,0.00000,9.604906,otm
99,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,p,2058.690790,2025-02-12 00:00:00+00:00,4247.351996,60.36,95757.81054,4242.18946,-2183.498670,itm
100,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,p,2065.054790,2025-02-12 00:00:00+00:00,3951.033792,62.78,96049.03000,3950.97000,-1885.915210,itm
102,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,101000.0,c,9.575306,2025-02-12 00:00:00+00:00,1.639292,63.79,95757.81054,0.00000,9.575306,otm
...,...,...,...,...,...,...,...,...,...,...,...,...
6607,2025-02-14 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4241.673255,2025-03-07 00:00:00+00:00,4883.122487,47.01,97944.45750,1055.54250,3186.130755,itm
6608,2025-02-15 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4733.167865,2025-03-07 00:00:00+00:00,4682.831357,46.34,98009.49750,990.50250,3742.665365,itm
6609,2025-02-16 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4952.941430,2025-03-07 00:00:00+00:00,5402.500208,46.74,96553.14000,2446.86000,2506.081430,itm
6610,2025-02-17 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,5459.360820,2025-03-07 00:00:00+00:00,5608.945391,46.71,96022.42000,2977.58000,2481.780820,itm


In [12]:
df_opt_chain = btc.chain.select_chain()
df_opt_chain

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
1539,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,100000.0,c,9.619789,2025-02-18 00:00:00+00:00,0.909070,54.54,96201.093864,0.000000,9.619789,otm
1543,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,100000.0,p,3131.241320,2025-02-18 00:00:00+00:00,3799.688712,54.54,96201.093864,3798.906136,-667.664816,itm
1547,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,101000.0,c,9.619789,2025-02-18 00:00:00+00:00,0.221255,59.66,96201.093864,0.000000,9.619789,otm
1551,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,101000.0,p,9.619789,2025-02-18 00:00:00+00:00,4798.967382,59.66,96201.093864,4798.906136,-4789.286347,itm
1555,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,102000.0,c,9.619789,2025-02-18 00:00:00+00:00,0.021164,60.68,96201.093864,0.000000,9.619789,otm
...,...,...,...,...,...,...,...,...,...,...,...,...
1703,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,97500.0,p,1688.272970,2025-02-18 00:00:00+00:00,1317.063590,33.20,96201.093864,1298.906136,389.366834,itm
1707,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,98000.0,c,9.619789,2025-02-18 00:00:00+00:00,7.241777,36.66,96201.093864,0.000000,9.619789,otm
1711,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,98000.0,p,2020.155690,2025-02-18 00:00:00+00:00,1806.087487,36.66,96201.093864,1798.906136,221.249554,itm
1715,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,99000.0,c,9.619789,2025-02-18 00:00:00+00:00,4.057627,49.17,96201.093864,0.000000,9.619789,otm


Futures types:
* Currency
* Index
* Commodities
* Stock
* Percent

Is crypto new type or currency?


BRN specification https://www.cmegroup.com/markets/energy/crude-oil/brent-crude-oil.html

Option chain https://www.barchart.com/futures/quotes/CB*0/options?futuresOptionsView=merged

In [13]:
atm_nearest_strikes = btc.chain.get_atm_nearest_strikes()
atm_strike = atm_nearest_strikes[0]
atm_strike

np.float64(96000.0)

In [15]:
atm_strike_df = df_opt_chain[df_opt_chain['strike'] == atm_strike]
atm_strike_row = atm_strike_df[atm_strike_df['option_type'] == OptionType.CALL.code].iloc[0]
atm_strike_df

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
1683,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,96000.0,c,211.635358,2025-02-18 00:00:00+00:00,382.591514,28.13,96201.093864,201.093864,10.541494,atm
1687,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,96000.0,p,250.114514,2025-02-18 00:00:00+00:00,181.504255,28.13,96201.093864,0.000000,250.114514,atm


In [20]:
df_desk = btc.chain.get_desk(
    option_columns=['price', 'exchange_price', 'exchange_iv','price_status',
                    'intrinsic_value', 'time_value', 'timestamp', 'expiration_date', 'strike'])
res_col = ['time_value_call', 'intrinsic_value_call', 'exchange_iv_call', 'exchange_price_call', 'price_call', 'price_status_call',
            'strike',
           'price_status_put', 'price_put', 'exchange_price_put', 'exchange_iv_put', 'intrinsic_value_put', 'time_value_put',
           'timestamp', 'underlying_price', 'expiration_date', 'underlying_expiration_date']
df_desk[df_desk['strike'].isin(atm_nearest_strikes[:10])][res_col]

,time_value_call,intrinsic_value_call,exchange_iv_call,exchange_price_call,price_call,price_status_call,strike,price_status_put,price_put,exchange_price_put,exchange_iv_put,intrinsic_value_put,time_value_put,timestamp,underlying_price,expiration_date,underlying_expiration_date
5,-565.729734,2201.093864,39.90,2204.347714,1635.364130,itm,94000.0,otm,19.239578,3.327485,39.90,0.000000,19.239578,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00
6,703.853386,1701.093864,36.52,1709.509616,2404.947250,itm,94500.0,otm,9.619789,8.472148,36.52,0.000000,9.619789,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00
7,-383.411799,1201.093864,34.45,1227.811187,817.682065,itm,95000.0,otm,38.479156,26.757443,34.45,0.000000,38.479156,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00
8,-172.005469,701.093864,28.03,752.876432,529.088395,itm,95500.0,otm,57.718734,51.806412,28.03,0.000000,57.718734,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00
9,10.541494,201.093864,28.13,382.591514,211.635358,atm,96000.0,atm,250.114514,181.504255,28.13,0.000000,250.114514,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00
10,125.057257,0.000000,29.18,156.522625,125.057257,otm,96500.0,itm,480.989450,455.419089,29.18,298.906136,182.083314,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00
11,38.479156,0.000000,30.98,55.071368,38.479156,otm,97000.0,itm,1587.265185,853.950593,30.98,798.906136,788.359049,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00
12,9.619789,0.000000,33.20,18.201603,9.619789,otm,97500.0,itm,1688.272970,1317.063590,33.20,1298.906136,389.366834,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00
13,9.619789,0.000000,36.66,7.241777,9.619789,otm,98000.0,itm,2020.155690,1806.087487,36.66,1798.906136,221.249554,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00
14,9.619789,0.000000,49.17,4.057627,9.619789,otm,99000.0,itm,3463.124040,2802.870784,49.17,2798.906136,664.217904,2025-02-18 00:00:00+00:00,96201.093864,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00


The time value of the option decreases by the expiration date

ITM options have a low time value (low weight) in the total option price. In fact, the deeper an option is in the money, the lower its time value and the more it resembles a linear underlying asset (i.e., the corresponding futures).

OTM options have no intrinsic value (which is understandable – they are unprofitable to execute). The entire value of an out–of-money option is the intrinsic value. It is important to understand that if the market does not overcome the strike of this option, the final expiration option price will be zero.

ATM options have the maximum weight of the time value in the option price and the maximum non-linearity. For this reason, they are the most liquid on the market. When the underlying asset moves up or down from current levels, the liquidity of these options will decrease.

In [21]:
settlement_date, expiration_date = btc.chain.get_settlement_and_expiration_date()
settlement_date, expiration_date

(Timestamp('2025-02-18 00:00:00+0000', tz='UTC'),
 Timestamp('2025-02-18 00:00:00+0000', tz='UTC'))

In [22]:
btc.chart.init()
btc.chart.price.time_values()
btc.chart.show()

In [23]:
btc.chart.price.time_values_for_strike(strike=atm_nearest_strikes[5])
btc.chart.price.time_values_for_distance(distance=0)
btc.chart.show()

In [24]:
df_time_value_strike = btc.analytic.price.time_value_series_by_strike_to_atm_distance(strike=atm_nearest_strikes[5])
df_time_value_strike.iloc[[0, int(len(df_time_value_strike) / 2), -1]]

,timestamp,strike,time_value
0,2025-02-15 00:00:00+00:00,100000.0,351.327924
31,2025-02-16 00:00:00+00:00,97000.0,577.041720
68,2025-02-18 00:00:00+00:00,97500.0,9.619789


In [25]:
df_time_value_atm = btc.analytic.price.time_value_series_by_atm_distance(distance=0)
df_time_value_atm

,timestamp,strike,time_value
20,2025-02-15 00:00:00+00:00,98000.0,878.319810
27,2025-02-16 00:00:00+00:00,96000.0,765.613946
47,2025-02-17 00:00:00+00:00,96000.0,210.712172
77,2025-02-18 00:00:00+00:00,96000.0,10.541494


In [26]:
btc.chart.init()
btc.chart.price.time_values([df_time_value_strike, df_time_value_atm], [f'Strike cur ATM {atm_strike}', 'ATM'])
btc.chart.show()

## Non-linearity of options

In [27]:
strikes = btc.chain.get_atm_nearest_strikes()
strike_start = sorted(filter(lambda x: x <= atm_strike * 0.95, strikes))[-1]
strike_end = sorted(filter(lambda x: x >= atm_strike * 1.1, strikes))[0]
strike_start, strike_end

(np.float64(90000.0), np.float64(106000.0))

In [28]:
fut_legs = [OptionLeg(strike=0, lots=1, type=LegType.FUTURE)]
fut_risk_profile, fut_risk_legs_pnl = btc.analytic.risk.chain_risk_profile(fut_legs)
fut_risk_profile = fut_risk_profile.loc[strike_start:strike_end]
fut_risk_profile.head(2)

,risk_pnl
strike,
90000.0,-6201.093864
92000.0,-4201.093864


In [29]:
call_legs = [OptionLeg(strike=atm_strike, lots=1, type=LegType.OPTION_CALL)]
call_risk_profile, call_risk_legs_pnl = btc.analytic.risk.chain_risk_profile(call_legs)
call_risk_profile = call_risk_profile.loc[strike_start:strike_end]
call_risk_profile.head(2)

,risk_pnl
strike,
90000.0,-211.635358
92000.0,-211.635358


In [30]:
call_legs_1 = [OptionLeg(strike=strikes[15], lots=1, type=LegType.OPTION_CALL)]
call_risk_profile_1, _ = btc.analytic.risk.chain_risk_profile(call_legs_1)
call_risk_profile_1 = call_risk_profile_1.loc[strike_start:strike_end]
call_risk_profile_1.head(2)

,risk_pnl
strike,
90000.0,-9.619789
92000.0,-9.619789


In [31]:
data = [
    go.Scatter(x=fut_risk_profile.index.to_list(), y=fut_risk_profile['risk_pnl'].to_list(), mode='lines', name='fut'),
    go.Scatter(x=call_risk_profile.index.to_list(), y=call_risk_profile['risk_pnl'].to_list(), mode='lines',
               name=f'call atm {call_legs[0].strike}'),
    go.Scatter(x=call_risk_profile_1.index.to_list(), y=call_risk_profile_1['risk_pnl'].to_list(), mode='lines',
               name=f'call {call_legs_1[0].strike}')
]  # markers, lines
iplot(data)